<a href="https://colab.research.google.com/github/u21598012/COS-760-EA/blob/BERT-fine-tuned/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 21.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
import optuna
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM,
    AutoModel,
    AutoConfig,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    AutoModelForSequenceClassification, get_scheduler
)
from peft import LoraConfig, get_peft_model
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('processed_data_it3.csv')
df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise,emotions,tokens,stemmed_tokens
0,hau_train_track_a_00001,kotu ta yi hukunci kan shari'ar zaben dan maja...,0,0,0,0,0,1,"('surprise',)","['kotu', 'hukunci', 'kan', ""shari'ar"", 'zaben'...","['kotu', 'hukunci', 'kan', ""shari'ar"", 'zaben'..."
1,hau_train_track_a_00002,"toh fah inji 'yan magana suka ce """"""""ana wata ...",0,0,0,0,0,1,"('surprise',)","['toh', 'fah', 'inji', ""'yan"", 'magana', '``',...","['toh', 'fah', 'inji', ""'yan"", 'magana', ""''"",..."
2,hau_train_track_a_00003,bincike ya nuna yan najeriya sun fi damuwa da ...,0,0,1,0,1,0,"('fear', 'sadness')","['bincike', 'nuna', 'yan', 'najeriya', 'fi', '...","['bincik', 'nuna', 'yan', 'najeriya', 'fi', 'd..."
3,hau_train_track_a_00004,kwamishina ya musanta rahoton masari ya cire k...,0,0,0,0,0,0,(),"['kwamishina', 'musanta', 'rahoton', 'masari',...","['kwamishina', 'musanta', 'rahoton', 'masari',..."
4,hau_train_track_a_00005,innalillahi wa inna ilaihir raji'un: allah ya ...,0,0,0,0,1,0,"('sadness',)","['innalillahi', 'wa', 'inna', 'ilaihir', ""raji...","['innalillahi', 'wa', 'inna', 'ilaihir', ""raji..."


In [4]:
text = df['stemmed_tokens']
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
labels = df[emotion_labels].values.tolist()
df[emotion_labels] = df[emotion_labels].astype(int)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Davlan/bert-base-multilingual-cased-finetuned-hausa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df[emotion_labels].values, test_size=0.2, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-hausa")


In [7]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.texts)


In [9]:
train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
val_dataset = EmotionDataset(val_texts, val_labels, tokenizer)

In [8]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
        "Davlan/bert-base-multilingual-cased-finetuned-hausa",
        num_labels=len(emotion_labels),
        problem_type="multi_label_classification"
    )
# model = AutoModelForSequenceClassification.from_pretrained(
#     "Davlan/bert-base-multilingual-cased-finetuned-hausa",
#     num_labels=len(emotion_labels),
#     problem_type="multi_label_classification"
# )

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_strategy="no",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    optim="adamw_torch",
    seed=3,
    learning_rate=1.6788168340923137e-05,)

# training_args = TrainingArguments(
#     output_dir="./optuna_results", # Use a different output directory for tuning
#     per_device_train_batch_size=4, # You might tune this
#     per_device_eval_batch_size=16,  # You might tune this
#     save_strategy="no",
#     logging_dir='./optuna_logs',    # Use a different logging directory
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1", # Specify the metric to optimize
#     greater_is_better=True, # Specify if a higher metric is better
#     learning_rate=1.6788168340923137e-05,
#     num_train_epochs=4,
#     seed=3
# )

# training_args = TrainingArguments(
#     output_dir="./optuna_results", # Use a different output directory for tuning
#     per_device_train_batch_size=16, # You might tune this
#     per_device_eval_batch_size=16,  # You might tune this
#     save_strategy="no",
#     logging_dir='./optuna_logs',    # Use a different logging directory
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1", # Specify the metric to optimize
#     greater_is_better=True # Specify if a higher metric is better
# )



In [11]:
def compute_metrics(pred):
    logits, labels = pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int()
    labels = torch.tensor(labels).int()
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_micro': f1_score(labels, preds, average='micro'),
        'f1_samples': f1_score(labels, preds, average='samples'),
        'accuracy': (preds == labels).float().mean().item(),
    }

def custom_loss(outputs, labels):
    return BCEWithLogitsLoss()(outputs.logits, labels)

In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init )

# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="optuna",
#     n_trials=20
# )

# print("Best trial:")
# print(best_trial)

trainer.train()

metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

# Save the trained model and tokenizer
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-hausa and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.598000
20,0.515900
30,0.528400
40,0.487500
50,0.442500
60,0.458000
70,0.478800
80,0.435400
90,0.435000
100,0.443900


In [ ]:
print("Number of records : ", len(df))

count_anger = df['anger'].value_counts().get(1, 0)
print("Occurrences of 'anger':", count_anger)

count_disgust = df['disgust'].value_counts().get(1, 0)
print("Occurrences of 'disgust':", count_disgust)

count_fear = df['fear'].value_counts().get(1, 0)
print("Occurrences of 'fear':", count_fear)

count_joy = df['joy'].value_counts().get(1, 0)
print("Occurrences of 'joy':", count_joy)

count_sadness = df['sadness'].value_counts().get(1, 0)
print("Occurrences of 'sadness':", count_sadness)

count_surprise = df['surprise'].value_counts().get(1, 0)
print("Occurrences of 'surprise':", count_surprise)


In [2]:
!pip freeze > requirements.txt